In [1]:
import os, warnings

warnings.filterwarnings(action='ignore')

In [2]:
import tensorflow as tf
import numpy as np
from keras.utils import np_utils

categories = ["top", "pants","skirts"]
nb_classes = len(categories)

# 데이터 불러오기
def load_dataset():
  x_train, x_test, y_train, y_test = np.load("clothes_2.npy", allow_pickle=True)
  x_train = x_train.astype("float") / 256
  x_test = x_test.astype("float") / 256
  y_train = np_utils.to_categorical(y_train, nb_classes)
  y_test = np_utils.to_categorical(y_test, nb_classes)
  return x_train, x_test, y_train, y_test

X_train, X_test, y_train, y_test = load_dataset()

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Using TensorFlow backend.


(1758, 64, 64, 3)
(1758, 3)
(586, 64, 64, 3)
(586, 3)


In [0]:
# import keras

# y_train = keras.utils.to_categorical(y_train, nb_classes)
# y_test = keras.utils.to_categorical(y_test, nb_classes)

# print(y_train.shape)
# print(y_test.shape)

In [6]:
X = tf.placeholder(tf.float32, [None, 64, 64, 3])
Y = tf.placeholder(tf.float32, [None, 3])
keep_prob = tf.placeholder(tf.float32)

In [7]:
print(X)

Tensor("Placeholder_3:0", shape=(?, 64, 64, 3), dtype=float32)


In [8]:
W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

print(W1)
print(L1)

L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L1)

<tf.Variable 'Variable_1:0' shape=(3, 3, 3, 32) dtype=float32_ref>
Tensor("Relu_1:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 32, 32, 32), dtype=float32)


In [9]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)

print(W2)
print(L2)

L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L2)

<tf.Variable 'Variable_2:0' shape=(3, 3, 32, 64) dtype=float32_ref>
Tensor("Relu_2:0", shape=(?, 32, 32, 64), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 16, 16, 64), dtype=float32)


In [10]:
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)

print(W3)
print(L3)

L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='SAME')
print(L3)

<tf.Variable 'Variable_3:0' shape=(3, 3, 64, 128) dtype=float32_ref>
Tensor("Relu_3:0", shape=(?, 16, 16, 128), dtype=float32)
Tensor("MaxPool_3:0", shape=(?, 16, 16, 128), dtype=float32)


In [11]:
W4 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev=0.01))
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding='SAME')
L4 = tf.nn.relu(L4)

print(W4)
print(L4)

L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='SAME')
print(L4)

<tf.Variable 'Variable_4:0' shape=(3, 3, 128, 256) dtype=float32_ref>
Tensor("Relu_4:0", shape=(?, 16, 16, 256), dtype=float32)
Tensor("MaxPool_4:0", shape=(?, 16, 16, 256), dtype=float32)


In [12]:
W5 = tf.Variable(tf.random_normal([16 * 16 * 256, 512], stddev=0.01))
L5 = tf.reshape(L4, [-1, 16 * 16 * 256])
L5 = tf.matmul(L5, W5)
L5 = tf.nn.relu(L5)
L5 = tf.nn.dropout(L5, keep_prob)

print(W5)
print(L5)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
<tf.Variable 'Variable_5:0' shape=(65536, 512) dtype=float32_ref>
Tensor("dropout/mul_1:0", shape=(?, 512), dtype=float32)


In [13]:
W6 = tf.Variable(tf.random_normal([512, 3], stddev=0.01))

print(W6)

model = tf.matmul(L5, W6)

print(model)

<tf.Variable 'Variable_6:0' shape=(512, 3) dtype=float32_ref>
Tensor("MatMul_1:0", shape=(?, 3), dtype=float32)


In [14]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,
labels=Y))

optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [15]:
init = tf.global_variables_initializer() # 초기화
sess = tf.Session() # 세션 연결
sess.run(init) # 세션 초기화

In [16]:
batch_size = 100

total_batch = int(X_train.shape[0]/ batch_size)
print(total_batch)

epochs = 20

17


In [17]:
# 다음 배치를 읽어오기 위한 next_batch 유틸리티 함수를 정의합니다.
def next_batch(start, num, data, labels):
    data_X = data[start:start+num]
    data_y = labels[start:start+num]
    return np.asarray(data_X), np.asarray(data_y)

In [18]:
print(X_train.shape, y_train.shape)

batch_X, batch_y = next_batch(0, 10, X_train, y_train)
print(batch_X.shape, batch_y.shape)

(1758, 64, 64, 3) (1758, 3)
(10, 64, 64, 3) (10, 3)


In [19]:
total_cost = 0

print("전체 입력 데이터 : {}".format(X_train.shape))
print("전체 출력 데이터 : {}".format(y_train.shape))

for epoch in range(epochs):
    total_cost = 0
  
    for i in range(total_batch):
        batch_Xs, batch_ys = next_batch(batch_size*i, batch_size, X_train, y_train)
    
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_Xs, Y: batch_ys, keep_prob: 0.2})
        total_cost += cost_val
    
        if (i==0 or i==total_batch-1):
              print('data_step = {}, Avg. cost = {:.3f}'.format(i, cost_val))

    print('epoch: {} total.cost = {:.3f}'.format(epoch, total_cost))

전체 입력 데이터 : (1758, 64, 64, 3)
전체 출력 데이터 : (1758, 3)
data_step = 0, Avg. cost = 1.099
data_step = 16, Avg. cost = 0.744
epoch: 0 total.cost = 14.265
data_step = 0, Avg. cost = 0.726
data_step = 16, Avg. cost = 0.710
epoch: 1 total.cost = 12.159
data_step = 0, Avg. cost = 0.708
data_step = 16, Avg. cost = 0.702
epoch: 2 total.cost = 11.938
data_step = 0, Avg. cost = 0.701
data_step = 16, Avg. cost = 0.698
epoch: 3 total.cost = 11.847
data_step = 0, Avg. cost = 0.691
data_step = 16, Avg. cost = 0.701
epoch: 4 total.cost = 11.813
data_step = 0, Avg. cost = 0.692
data_step = 16, Avg. cost = 0.702
epoch: 5 total.cost = 11.684
data_step = 0, Avg. cost = 0.675
data_step = 16, Avg. cost = 0.686
epoch: 6 total.cost = 11.441
data_step = 0, Avg. cost = 0.695
data_step = 16, Avg. cost = 0.548
epoch: 7 total.cost = 10.631
data_step = 0, Avg. cost = 0.564
data_step = 16, Avg. cost = 0.396
epoch: 8 total.cost = 8.682
data_step = 0, Avg. cost = 0.488
data_step = 16, Avg. cost = 0.356
epoch: 9 total.cos

In [20]:
print(X_test.shape, y_test.shape)

(586, 64, 64, 3) (586, 3)


In [21]:
model

<tf.Tensor 'MatMul_1:0' shape=(?, 3) dtype=float32>

In [22]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('정확도:', sess.run(accuracy, feed_dict={X: X_train, Y: y_train, keep_prob: 1.}))

정확도: 0.9596132


In [23]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('정확도:', sess.run(accuracy, feed_dict={X: X_test, Y: y_test, keep_prob: 1.}))

정확도: 0.8976109
